In [19]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.3.1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.2
numpy 1.18.4
pandas 1.1.4
sklearn 0.23.2
tensorflow 2.3.1
tensorflow.keras 2.4.0


In [20]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu"))
model.add(keras.layers.AlphaDropout(rate=0.5))
# model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(0.01),
              metrics = ["accuracy"])

In [5]:
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.6758 - accuracy: 0.7714 - val_loss: 0.6705 - val_accuracy: 0.8474
Epoch 2/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4560 - accuracy: 0.8436 - val_loss: 0.5697 - val_accuracy: 0.8522
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3994 - accuracy: 0.8588 - val_loss: 0.5426 - val_accuracy: 0.8684
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3728 - accuracy: 0.8676 - val_loss: 0.5325 - val_accuracy: 0.8656
Epoch 5/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.3503 - accuracy: 0.8741 - val_loss: 0.5296 - val_accuracy: 0.8734
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3336 - accuracy: 0.8802 - val_loss: 0.4695 - val_accuracy: 0.8740
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3186 - accuracy: 0.8860 - val_loss: 0.4787 - val_accuracy:

## Save to SavedModel

In [8]:
tf.saved_model.save(model, "./keras_saved_graph")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_saved_graph/assets


In [9]:
# 查看 SavedModel
!saved_model_cli show --dir ./keras_saved_graph --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_20'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [12]:
!saved_model_cli show --dir ./keras_saved_graph \
    --tag_set serve --signature_def serving-default

The given SavedModel SignatureDef contains the following input(s):
The given SavedModel SignatureDef contains the following output(s):
Method name is: 


In [13]:
!saved_model_cli run --dir ./keras_saved_graph --tag_set server \
    --signature_def serving-default \
    --input_exprs 'flatten_input=np.ones((2, 28, 28))'

Traceback (most recent call last):
  File "/home/meprint/anaconda3/envs/xtest_gpu/bin/saved_model_cli", line 11, in <module>
    sys.exit(main())
  File "/home/meprint/anaconda3/envs/xtest_gpu/lib/python3.6/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/home/meprint/anaconda3/envs/xtest_gpu/lib/python3.6/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 643, in run
    init_tpu=args.init_tpu, tf_debug=args.tf_debug)
  File "/home/meprint/anaconda3/envs/xtest_gpu/lib/python3.6/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 303, in run_saved_model_with_feed_dict
    tag_set)
  File "/home/meprint/anaconda3/envs/xtest_gpu/lib/python3.6/site-packages/tensorflow_core/python/tools/saved_model_utils.py", line 120, in get_meta_graph_def
    ' could not be found in SavedModel')
RuntimeError: MetaGraphDef associated with tag-set server could not be found in SavedModel


In [14]:
loaded_saved_model = tf.saved_model.load('./keras_saved_graph')
print(list(loaded_saved_model.signatures.keys()))

['serving_default']


In [16]:
inference = loaded_saved_model.signatures['serving_default']
print(inference)
print(inference.structured_outputs)

<ConcreteFunction signature_wrapper(flatten_input) at 0x7F93F4B63B70>
{'dense_20': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_20')}


In [23]:
results = inference(tf.constant(x_test_scaled[0:1]))
print(results['dense_20'])

tf.Tensor(
[[1.8807198e-19 1.6721442e-15 4.0438728e-20 1.6282989e-18 2.9771683e-23
  2.0114485e-10 9.8699078e-21 7.2208439e-09 3.1765011e-16 1.0000000e+00]], shape=(1, 10), dtype=float32)
